## Week 2 Day 3

### Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails
    2 inpot guardrails - 
        - is personal name
        - is dollar
    1 outout guardrail
         - is math
         based on https://openai.github.io/openai-agents-python/guardrails/


In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, output_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel
import time
from datetime import datetime

In [ ]:
load_dotenv(override=True)

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:

import requests
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    print(payload)
    response=requests.post(pushover_url, data=payload)
    print(response)

push("pudhover test")

### It's easy to use any models with OpenAI compatible endpoints

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [ ]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
open_ai_client = AsyncOpenAI(api_key=openai_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)
open_ai_model = OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=open_ai_client)

sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3 =  Agent(name="Llama3.3 Sales Agent", instructions=instructions3, model=llama3_3_model)


In [ ]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

SEnd HTML email calls pushover as cant get a sendgrid key

In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    # mail = Mail(from_email, to_email, subject, content).get()
    # response = sg.client.mail.send.post(request_body=mail)
    push(f"test {from_email} , {to_email}")
    return {"status": "success"}

Test the HTML send 

In [ ]:
def send_html_email2(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(FROM_EMAIL)  # Change to your verified sender
    to_email = To(TO_EMAIL)  # Change to your recipient
    content = Content("text/html", html_body)
    # mail = Mail(from_email, to_email, subject, content).get()
    # response = sg.client.mail.send.post(request_body=mail)
    push(f"email (FROM_EMAIL) , {TO_EMAIAL} subject = {subject}")
    return {"status": "success"}

send_html_email2("test", "test")


In [ ]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [ ]:
email_tools = [subject_tool, html_tool, send_html_email]

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [ ]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."



** Guardrails

Input
1. Check if email request contains a personal name
2. Check if email request contains $ sign
Output
1. Check if output contains maths answer



1 Guardrail intput - trip if name check trios

In [ ]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check:Input Guardrail",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [ ]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    push(f"Name input guradrail:{is_name_in_message}")
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

Guardrail 2 - trip on $ in input

In [ ]:
class DollarSignCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

In [ ]:
guardrail_agent2 = Agent( 
    name="Dollar check",
    instructions="Check if there are any monertary values in the input message",
    output_type=DollarSignCheckOutput,
    model="gpt-4o-mini"
)

@input_guardrail
async def guardrail_against_dollar(ctx, agent, message):
    result = await Runner.run(guardrail_agent2, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    push(f"dollar guardrail {is_name_in_message}")
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

Output Guardrails
1. Check if there is any Math output
Based on :
https://openai.github.io/openai-agents-python/guardrails/

In [ ]:
from agents import Agent, Runner, RunContextWrapper, output_guardrail, GuardrailFunctionOutput

class MessageOutput(BaseModel): 
    response: str

class MathOutput(BaseModel): 
    reasoning: str
    is_math: bool

guardrail_agent_output = Agent(
    name="Math Guardrail check",
    instructions="Check if the output includes any math formulas.",
    output_type=MathOutput,
    model="gpt-4o-mini"
)

@output_guardrail
async def math_guardrail(  
    ctx: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    # result = await Runner.run(reguardrail_agent_output, output.response, context=ctx.context)
    result = await Runner.run(guardrail_agent_output, output, context=ctx.context)
    push(f"Math result = {result.final_output.is_math}")
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math,
    )


# End of output tripwire


In [ ]:
# Output Guardrail - IS austrailian city mentioned


# Message output defined already
# throw s a tyoe error to do with is_math so commenteed out

""" ""
class OzOutput(BaseModel): 
    reasoning: str
    is_oz_city: bool
    
guardrail_oz_agent_output = Agent(
    name="Oz Guardrail check",
    instructions="Check if the output includes any Australian towns or cities.",
    output_type=OzOutput,
    model="gpt-4o-mini"
)
print("type:",type(guardrail_oz_agent_output))
@output_guardrail
async def oz_output_guardrail(  
    ctx: RunContextWrapper, agent: Agent, output: OzOutput
) -> GuardrailFunctionOutput:
    # result = await Runner.run(guardrail_agent_output, output.response, context=ctx.context)
#    result = await Runner.run(guardrail_oz_agent_output = Agent(, output, context=ctx.context)

    result = await Runner.run(gguardrail_oz_agent_output,output, context=ctx.context)
    push(f"Oz result = {result.final_output.is_math}")
    print(f"Oz result = {result.final_output.is_math}")
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math
    )
"""


In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name,guardrail_against_dollar], 
    # output_guardrails=[math_guardrail,oz_output_guardrail]
    output_guardrails=[math_guardrail]
    )

# Input Tripwire 1 : NAmne trip wire
# message = "Send out a cold sales email addressed to Dear CEO from for Alice"

# Input Tripwire 2 :  tripwire
# message = "Send out a cold sales email addressed to Dear CEO from for $1000"

# Output tripwire :math tripwire\
# message = "solve for x in 2x+2=10 with reasoning"

# Output tripwire :Australian city
# message = "What is the largest australian city"
message="what words ryhme with pine"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)
    print("result=",result)

## Check out the trace:

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>